In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cv2
import os
import tensorflow as tf
from tensorflow import keras

# Test Images

In [ ]:
model = keras.models.load_model('emotions_classifer_2')

emotion = {0: 'angry', 1: 'disgust', 2: 'fear', 3: 'happy', 4: 'sad', 5: 'surprise', 6: 'neutral'}

def test_images(model, nbr_images_folder=2, emotion_dict=emotion):
    path = os.getcwd()+'/dataset/test'
    plt.figure(figsize=(18, 8))
    num_axe = 1 

    for folder in os.listdir(path):
        path_2 = path+f'/{folder}'
        label_true = folder
        for i, image in enumerate(os.listdir(path_2)):
            if i == nbr_images_folder:
                break

            img = cv2.imread(path_2+f'/{image}', 0)
            ax = plt.subplot(2,7,num_axe)
            num_axe += 1

            img_to_pred = img
            img_to_pred = img_to_pred.reshape(1, 48, 48, 1)
            img_to_pred = img_to_pred.astype("float32")
            img_to_pred /= 255
            prediction = (model.predict(img_to_pred) > 0.5).astype("int32")
            prediction = prediction[0]
            try:
                pred = np.where(prediction == 1)[0][0]
                pred_text = emotion_dict[pred]
            except:
                break
            ax.imshow(img, cmap='gray')
            ax.set_title(f'True:{label_true}, Pred:{pred_text}')
            ax.set_xticks([])
            ax.set_yticks([])
            plt.tight_layout()

In [ ]:
test_images(model)

# Test Vidéo

In [5]:
face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

In [3]:
cap = cv2.VideoCapture(0)

while(True):
    # Capture frame-by-frame
    ret, frame = cap.read()
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    
    # Detect face
    faces = face_cascade.detectMultiScale(gray, 1.3, 5)

    for (x,y,w,h) in faces:
        cv2.rectangle(frame,(x,y),(x+w,y+h),(255,0,0),2)
        frame_to_predict = gray[y:y+h, x:x+w]
        frame_to_predict = cv2.resize(frame_to_predict, (48, 48))
        frame_to_predict = np.array(frame_to_predict)
        frame_to_predict = frame_to_predict.reshape(1, 48, 48, 1)
        frame_to_predict = frame_to_predict.astype("float32")
        frame_to_predict /= 255
    
        try:
            prediction = (model.predict(frame_to_predict) > 0.5).astype("int32")
            prediction = prediction[0]
            pred = np.where(prediction == 1)[0][0]
            pred_text = emotion[pred]
            cv2.putText(frame, pred_text, (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (255,0,0), 2)
        except:
            continue
    
    cv2.imshow('frame',frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# When everything done, release the capture
cap.release()
cv2.destroyAllWindows()